In [1]:
!pip freeze | grep scikit-learn


scikit-learn==1.6.1


In [2]:
!python -V


Python 3.12.1


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')


In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [10]:
print(y_pred.std())

6.247488852238703


In [ ]:
year = 2023
month = 3
taxi_type = 'yellow'
output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'

In [ ]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [14]:
df['predicted_duration'] = y_pred

In [ ]:
def apply_model(df, y_pred, output_file):

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['predicted_duration'] = y_pred
    #df_result['tpep_pickup_datetime'] = df['tpep_pickup_datetime']
    #df_result['PULocationID'] = df['PULocationID']
    #df_result['DOLocationID'] = df['DOLocationID']
    #df_result['actual_duration'] = df['duration']
    
    #df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    #df_result['model_version'] = run_id
    
    df_result.to_parquet(
        output_file,
        engine='pyarrow',
        compression=None,
        index=False
    )

In [32]:
apply_model(df, y_pred, output_file)

In [ ]:
#!du -sh output/yellow/*

66M	output/yellow/2023-03.parquet


Q3. Creating the scoring script


#jupyter nbconvert --to script starter.ipynb

"hashes": [
                "sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c",  
            ],

Q6. Docker container

predicted mean duration: 0.19174419265916945

Q5. Parametrize the script


predicted mean duration: 14.292282936862449
